In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
# Daten laden
with open("data.json", encoding="utf8") as f:
    in_json = f.read()
js = json.loads(in_json)
data = js["data"][0]

# Struktur laden
with open("structure.json", encoding="utf8") as f:
    in_json = f.read()
structure = json.loads(in_json)

In [3]:
# Erstellt aus den json-Daten ein Pandas Dataframe
def data2df(data, lng="de"):
    # Daten als Numpy-Array einladen
    arr = np.array(data["value"]).reshape(data["size"])

    # Array flatten und Indices anspeichern
    flat_arr = []
    for idx, subarray in np.ndenumerate(arr):
        # Addiere den Index und den Wert als Tuple
        flat_arr.append((idx, subarray))

    # Liste zur Übersetzung der Indices in Codes erstellen
    dim = []
    for cat in data["id"]:
        ind = data["dimension"][cat]["category"]["index"]
        ind_rev = {value: key for key, value in ind.items()}
        dim.append(ind_rev)

    # Incdies in Codes/Werte übersetzen und Tabelle erstellen
    table = []
    for x in flat_arr:
        row = []
        for i, j in enumerate(x[0]):
            if not data["id"][i] in ["statistic", "content"]:
                code = dim[i][j]
                val = structure["variableValues"][code]["label"][lng]
                row.append(code)
                row.append(val)
            
        row.append(int(x[1]))
        table.append(row)

    # Spaltennamen erstellen
    cols = [item for sublist in [[col, structure["variables"][col]["label"][lng]] for col in data["id"] if not col in ["statistic", "content"]] for item in sublist] + ["Value"]
    
    # Tabelle als DataFrame zurückgeben
    return pd.DataFrame(table, columns=cols)

In [4]:
df = data2df(data)
df

,STAG,Stichtag,GES,Geschlecht,RECGL3,Ausgewählte Aufenthaltstitel,STAAG6,Staatsangehörigkeit,KREISE,Kreise,Value
0,2024-12-31,31.12.2024,GESM,männlich,REC-EU-FZKB,Aufenthaltsrecht nach FreizügG/EU,ST287,Ägypten,01001,"Flensburg, kreisfreie Stadt",0
1,2024-12-31,31.12.2024,GESM,männlich,REC-EU-FZKB,Aufenthaltsrecht nach FreizügG/EU,ST287,Ägypten,01002,"Kiel, kreisfreie Stadt",0
2,2024-12-31,31.12.2024,GESM,männlich,REC-EU-FZKB,Aufenthaltsrecht nach FreizügG/EU,ST287,Ägypten,01003,"Lübeck, kreisfreie Stadt",0
3,2024-12-31,31.12.2024,GESM,männlich,REC-EU-FZKB,Aufenthaltsrecht nach FreizügG/EU,ST287,Ägypten,01004,"Neumünster, kreisfreie Stadt",0
4,2024-12-31,31.12.2024,GESM,männlich,REC-EU-FZKB,Aufenthaltsrecht nach FreizügG/EU,ST287,Ägypten,01051,"Dithmarschen, Landkreis",0
...,...,...,...,...,...,...,...,...,...,...,...
2574203,2024-12-31,31.12.2024,GESW,weiblich,REC-AT-05,"Ohne Aufenthaltstitel, Duldung oder Gestattung",STAAT900,Ungeklärt / Ohne Angabe,16073,"Saalfeld-Rudolstadt, Landkreis",0
2574204,2024-12-31,31.12.2024,GESW,weiblich,REC-AT-05,"Ohne Aufenthaltstitel, Duldung oder Gestattung",STAAT900,Ungeklärt / Ohne Angabe,16074,Saale-Holzland-Kreis,0
2574205,2024-12-31,31.12.2024,GESW,weiblich,REC-AT-05,"Ohne Aufenthaltstitel, Duldung oder Gestattung",STAAT900,Ungeklärt / Ohne Angabe,16075,Saale-Orla-Kreis,0
2574206,2024-12-31,31.12.2024,GESW,weiblich,REC-AT-05,"Ohne Aufenthaltstitel, Duldung oder Gestattung",STAAT900,Ungeklärt / Ohne Angabe,16076,"Greiz, Landkreis",0


In [5]:
# Darstellung wie online
pd.pivot_table(
    df, 
    values=['Value'], 
    index=['KREISE', 'Staatsangehörigkeit'], 
    columns=['Ausgewählte Aufenthaltstitel','Geschlecht'], 
    aggfunc="sum"
)

Value  \
Ausgewählte Aufenthaltstitel Antrag auf einen Aufenthaltstitel gestellt   
Geschlecht                                                     männlich   
KREISE Staatsangehörigkeit                                                
01001  Afghanistan                                                   60   
       Albanien                                                      10   
       Algerien                                                       5   
       Andorra                                                        0   
       Angola                                                         0   
...                                                                 ...   
16077  Zypern                                                         0   
       Ägypten                                                        0   
       Äquatorialguinea                                               0   
       Äthiopien                                                      0   
       Österreich                                                     0   

                                                                      \
Ausgewählte Aufenthaltstitel          Aufenthaltsgestattung            
Geschlecht                   weiblich              männlich weiblich   
KREISE Staatsangehörigkeit                                             
01001  Afghanistan                 15                    25       15   
       Albanien                     5                     0        0   
       Algerien                     0                     0        5   
       Andorra                      0                     0        0   
       Angola                       0                     0        0   
...                               ...                   ...      ...   
16077  Zypern                       0                     0        0   
       Ägypten                      0                     0        0   
       Äquatorialguinea             0                     0        0   
       Äthiopien                    0                     0        0   
       Österreich                   0                     0        0   

                                                                         \
Ausgewählte Aufenthaltstitel Aufenthaltsrecht nach FreizügG/EU            
Geschlecht                                            männlich weiblich   
KREISE Staatsangehörigkeit                                                
01001  Afghanistan                                           5        0   
       Albanien                                              0        5   
       Algerien                                              0        0   
       Andorra                                               0        0   
       Angola                                                0        0   
...                                                        ...      ...   
16077  Zypern                                                0        0   
       Ägypten                                               0        0   
       Äquatorialguinea                                      0        0   
       Äthiopien                                             0        0   
       Österreich                                           15       15   

                                                                                 \
Ausgewählte Aufenthaltstitel Befristete AE, besondere Gründe und nationale Visa   
Geschlecht                                                             männlich   
KREISE Staatsangehörigkeit                                                        
01001  Afghanistan                                                           10   
       Albanien                                                               5   
       Algerien                                                               0   
       Andorra                                                                0   
       Angola                                                        